In [2]:
import openai as ai
import os
import json
import requests
import sys
import pandas as pd
from google.cloud import translate
from google.cloud import translate_v2 as translate
import pandas as pd
from openai import OpenAI
import itertools


**Inputing date from excel spreadsheat into pandas dataframe**

In [3]:
# read csv file
df = pd.read_csv('tibetan-definitions.csv', encoding='utf-8')
df.head(100)

,tibetan,wylie,other,definition
0,ལན་ཆགས།,lan chags/,lenchak,karmic debt ■ Karmic debt. A harmful effect th...
1,དག་པ་འཇིག་རྟེན་པའི་ཡེ་ཤེས,dag pa 'jig rten pa'i ye shes/,dakpa jikten peyeshé,དག་པ་འཇིག་རྟེན་པའི་ཡེ་ཤེས ■ A term used by Śān...
2,སྨན་བཅོས་སྐོར་བསྐྱོད,sman bcos skor bskyod/,menchö korkyö,སྨན་བཅོས་སྐོར་བསྐྱོད
3,སྨན་བཅོས་ཁང༌,sman bcos khang /,menchökhang,སྨན་བཅོས་ཁང༌ ■ clinic ■ clinic
4,སྨན་བཅོས་འཕྲོད་བསྟེན,sman bcos 'phrod bsten/,menchö tröten,སྨན་བཅོས་འཕྲོད་བསྟེན
...,...,...,...,...
95,སྨན་མོ་མཆེད་བདུན,sman mo mched bdun/,menmo chedün,སྨན་མོ་མཆེད་བདུན ■ Seven Menmo Sisters
96,སྨན་རྩི,sman rtsi/,mentsi,སྨན་རྩི ■ like ཁ་བཏགས་ a dyed ཁ་བཏགས་ with des...
97,སྨན་རྩིང༌,sman rtsing /,mentsing,སྨན་རྩིང༌
98,སྨན་རྩིས,sman rtsis/,mentsi,སྨན་རྩིས ■ [2167] སྨན་དཔྱད་དང་རྩིས་རིག་...


In [21]:
tibetan_column = df['tibetan'].head(10)
wylie_column = df['wylie'].head(10)
other_column = df['other'].head(10)
definitions_given_column = df['definition'].head(10)
# What type of data type is the columns?
print(type(tibetan_column))
# How do I convert the column to a list?
tibetan_column

<class 'pandas.core.series.Series'>


0                        ལན་ཆགས།
1      དག་པ་འཇིག་རྟེན་པའི་ཡེ་ཤེས
2           སྨན་བཅོས་སྐོར་བསྐྱོད
3                   སྨན་བཅོས་ཁང༌
4           སྨན་བཅོས་འཕྲོད་བསྟེན
5    སྨན་བཅོས་འཕྲོད་བསྟེན་སྡེ་ཁག
6                  སྨན་བཅོས་བྱེད
7                སྨན་བཅོས་བྱེད་པ
8                 སྨན་བཅོས་མི་སྣ
9                སྨན་བཅོས་ཡོ་བྱད
Name: tibetan, dtype: object

In [53]:
# merge the columns
new_df = df[['tibetan', 'wylie']].head(10)
new_df.head(10)

,tibetan,wylie
0,ལན་ཆགས།,lan chags/
1,དག་པ་འཇིག་རྟེན་པའི་ཡེ་ཤེས,dag pa 'jig rten pa'i ye shes/
2,སྨན་བཅོས་སྐོར་བསྐྱོད,sman bcos skor bskyod/
3,སྨན་བཅོས་ཁང༌,sman bcos khang /
4,སྨན་བཅོས་འཕྲོད་བསྟེན,sman bcos 'phrod bsten/
5,སྨན་བཅོས་འཕྲོད་བསྟེན་སྡེ་ཁག,sman bcos 'phrod bsten sde khag
6,སྨན་བཅོས་བྱེད,sman bcos byed/
7,སྨན་བཅོས་བྱེད་པ,sman bcos byed pa/
8,སྨན་བཅོས་མི་སྣ,sman bcos mi sna/
9,སྨན་བཅོས་ཡོ་བྱད,sman bcos yo byad/


# **OpenAI API**

In [99]:

from typing import List, Optional
def openai_translate(language_from: pd.Series, language_to: str, translate_column: pd.Series):

    client = OpenAI(api_key="openai_key")

    # prompt = f"create a english 1-3 word definition of  other text: {other_text}"
    prompt =  [
            {
                "role": "user",
                "content": f"""You are a translator. Translate the {language_from} to {language_to}. 
                Give ONLY the {language_to} translation in the form of a column:
                {translate_column}
                """
            }
        ]


    # Actually doing the chatgpt transtion meaning asking the question and getting response
    chat_completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=prompt,
        max_tokens=10  # Limit response to 1-3 words
    )
    defintions = chat_completion.choices[0].message.content
    return defintions

**OpenAI tranlating Function Testing**

In [100]:

output = openai_translate('tibetan', 'english', new_df)
output

'| English Translation                                |\n|---------------------------------------------------|\n|'

Index(['Unnamed: 0',
       'English Translation                                       ',
       'Unnamed: 2'],
      dtype='object')

In [101]:
data_io = StringIO(output)
output_df = pd.read_csv(data_io, sep='|' ,skipinitialspace=True, engine='python')
# Drop columns that not English
output_df = output_df[['English Translation                                       ']]
# Rename the column
output_df = output_df.rename(columns={'English Translation                                       ': 'definition'})
# Drop the first row
output_df = output_df.iloc[1:]
output_df.iloc[2]

KeyError: "None of [Index(['English Translation                                       '], dtype='object')] are in the [columns]"

: 

# **Google Translate API**

In [ ]:
import pandas as pd
from google.cloud import translate_v2 as translate

client = translate.Client.from_service_account_json('path/to/your/credentials.json')

# Sample large data
data = {'Tibetan Text': ["བོད་སྐད།"] * 1000}
df = pd.DataFrame(data)

# Batch translate function
def batch_translate(texts):
    results = client.translate(texts, target_language='en')
    return [result['translatedText'] for result in results]

# Chunk data and translate in batches
chunk_size = 100
chunks = [df['Tibetan Text'][i:i + chunk_size] for i in range(0, df.shape[0], chunk_size)]
translated_chunks = [batch_translate(chunk) for chunk in chunks]
df['English Translation'] = [item for sublist in translated_chunks for item in sublist]
print(df)


In [1]:
def translate_text(target: str, text: str) -> dict:

    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    if isinstance(text, bytes):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    print("Text: {}".format(result["input"]))
    print("Translation: {}".format(result["translatedText"]))
    print("Detected source language: {}".format(result["detectedSourceLanguage"]))

    return result



In [ ]:
def get_supported_languages(
    project_id: str = "YOUR_PROJECT_ID",
) -> translate.SupportedLanguages:
    client = translate.TranslationServiceClient()

    parent = f"projects/{project_id}"

    response = client.get_supported_languages(parent=parent)

    # List language codes of supported languages.
    print("Supported Languages:")
    for language in response.languages:
        print(f"Language Code: {language.language_code}")

    return response


In [ ]:
# Imports the Google Cloud Translation library
from google.cloud import translate


# Initialize Translation client
def translate_Google(text: str = "YOUR_TEXT_TO_TRANSLATE", project_id: str = "YOUR_PROJECT_ID") -> translate.TranslationServiceClient:
    

    client = translate.TranslationServiceClient()

    location = "global"

    parent = f"projects/{project_id}/locations/{location}"


    response = client.translate_text(
        request={
            # API Key for Google Translate API 
            "parent": parent,
            # Text to be translated
            "contents": [text],
            # Format to translate text
            "mime_type": "text/plain",  # mime types: text/plain, text/html
            # Language to tranlstate from
            "source_language_code": "en-US",
            # Language to translate text to
            "target_language_code": "en",
        }
    )

    # Display the translation for each input text provided
    for translation in response.translations:
        print(f"Translated text: {translation.translated_text}")

    return response


**Google Tranlate Function Testing**

In [2]:
print(translate_text("en", "བོད་སྐད།"))

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

# **HuggingFace API**

**Install the Hugging Face API**

In [4]:
# NOTE: "bo" is language code for Tibetan and "en" is language code for English.
# English is the default target language.
def translate(text: str, source_language: str, target_language: str = "en") -> str:
    from transformers import MarianMTModel, MarianTokenizer
    model_name = f"Helsinki-NLP/opus-mt-{source_language}-{target_language}"
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    
    # Perform the translation
    translated_tokens = model.generate(**inputs)
    
    # Decode the translated tokens
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    
    return translated_text


In [6]:
text = "Hola"
source = "es"
target = "en"
translation = translate(text, source, target)
translation


'Hello.'

**NOTE: translate_tibetan_to_english takes long time then doesn't even translate.**

In [7]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

def translate_tibetan_to_english(text: str) -> str:
    model_name = "facebook/mbart-large-50-many-to-many-mmt"
    model = MBartForConditionalGeneration.from_pretrained(model_name)
    tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
    
    # Set the source and target language
    tokenizer.src_lang = "bo"
    tokenizer.tgt_lang = "en"
    
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    
    # Perform the translation
    translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["en"])
    
    # Decode the translated tokens
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    
    return translated_text

# Example usage
tibetan_text = "བཀྲ་ཤིས་བདེ་ལེགས།"
translation = translate_tibetan_to_english(tibetan_text)
print(f"Tibetan to English: {translation}")


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

c:\Users\Elijah\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Elijah\.cache\huggingface\hub\models--facebook--mbart-large-50-many-to-many-mmt. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyError: 'en'

In [9]:
from googletrans import Translator

def translate_tibetan_to_english(text: str) -> str:
    translator = Translator()
    translation = translator.translate(text, src='es', dest='en')
    return translation.text

# Example usage
tibetan_text = "hola"
english_translation = translate_tibetan_to_english(tibetan_text)
print(english_translation)
# Needs fixing: Doesn't work with Tibetan text but does with other languages

hello


**Testing**

In [33]:
# List of Spanish sentences
spanish_sentences = [
    "Hola, ¿cómo estás?",
    "¿Cuál es tu nombre?",
    "Me gusta la programación.",
    "La inteligencia artificial es fascinante.",
    "Tengo que irme ahora.",
    "Gracias por tu ayuda.",
    "¿Dónde está la estación de tren?",
    "Hace mucho calor hoy.",
    "Quiero aprender más sobre ciencia de datos.",
    "¿Puedes recomendarme un buen libro?"
]

# Translate each sentence and print the result
for sentence in spanish_sentences:
    english_translation = translate(sentence)
    print(f"Spanish: {sentence}\nEnglish: {english_translation}\n")


Spanish: Hola, ¿cómo estás?
English: Hey, how are you?

Spanish: ¿Cuál es tu nombre?
English: What's your name?

Spanish: Me gusta la programación.
English: I like programming.

Spanish: La inteligencia artificial es fascinante.
English: Artificial intelligence is fascinating.

Spanish: Tengo que irme ahora.
English: I have to go now.

Spanish: Gracias por tu ayuda.
English: Thanks for your help.

Spanish: ¿Dónde está la estación de tren?
English: Where's the train station?

Spanish: Hace mucho calor hoy.
English: It's so hot today.

Spanish: Quiero aprender más sobre ciencia de datos.
English: I want to learn more about data science.

Spanish: ¿Puedes recomendarme un buen libro?
English: Can you recommend a good book for me?



**Summerization**

**Making it into a function to use for summerizing defintions into 1-3 words.**

In [19]:
def summerize(text: str):
    from transformers import pipeline

    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

    return summarizer(text, max_length=5, min_length=1, do_sample=False)
# How do you change the length of the summerized text?
# Answer: 

**Testing**

In [20]:
thing: str = "To tickle or chuck someone under the chin: Dar o hacer la mamola a algn To take someone for a ride: An informal meaning "
print(summarizer(thing))

Your max_length is set to 142, but your input_length is only 34. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)


[{'summary_text': 'To tickle or chuck someone under the chin: Dar o hacer la mamola a algn. To take someone for a ride: An informal meaning. To tickle someone under their chin is to take them on a ride. It is also an informal way to tickle a person.'}]


**NOTE:** The model Bart-Large-CNN was fine tuned to summerize articals to a few sentences. so the shortest the model will summerize is ONE sentence.

# **Summary Alternative**

In [13]:
def summary_short(text: str):
    from keybert import KeyBERT
    # Initialize the KeyBERT model
    kw_model = KeyBERT()

    # Extract keywords
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=3)
    return keywords

**Testing**

In [15]:
text = "To tickle or chuck someone under the chin "
summary_short(text)

[('tickle', 0.5564), ('chin', 0.5387), ('chuck', 0.4044)]

# **DataFrame to JSON File**

In [19]:
tibetan_dict = {
    "tibetan": "བོད་སྐད།",
    "wylie": "bod skad",
    "other": "Tibetan language",
    "definition": "The language spoken in Tibet"
}
json_object = []
json_object.append(tibetan_dict)
json_object.append(tibetan_dict)
json_object

[{'tibetan': 'བོད་སྐད།',
  'wylie': 'bod skad',
  'other': 'Tibetan language',
  'definition': 'The language spoken in Tibet'},
 {'tibetan': 'བོད་སྐད།',
  'wylie': 'bod skad',
  'other': 'Tibetan language',
  'definition': 'The language spoken in Tibet'}]

In [26]:
import json

In [22]:
def dataframe_to_json_file(df: pd.DataFrame):
    json_object = []
    for index, row in df.iterrows():
        tibetan = row["tibetan"]
        wylie = row["wylie"]
        other = row["other"]
        definition = row["definition"]
        tibetan_dict = {
            "tibetan": tibetan,
            "wylie": wylie,
            "other": other,
            "definition": definition
        }
        json_object.append(tibetan_dict)
    return json_object

In [28]:
json_object = dataframe_to_json_file(df)
json.dumps(json_object, indent=2)

'[\n  {\n    "tibetan": "\\u0f63\\u0f53\\u0f0b\\u0f46\\u0f42\\u0f66\\u0f0d",\n    "wylie": "lan chags/",\n    "other": "lenchak",\n    "definition": "karmic debt \\u25a0 Karmic debt. A harmful effect that returns to someone who performed a negative action. \\u25a0 In dialect, this can mean \\u201cdifficulty\\u201d or \\u201chardship\\u201d (dka\' tshegs); it can also indicate a \\u201cnegative condition\\u201d or a \\u201csource from which suffering arises\\u201d (chags sgo). \\u25a0 \\u0f63\\u0f53\\u0f0b\\u0f46\\u0f42\\u0f66\\u0f0d \\u25a0 a particular type of generalized karma which designates a relationship across lives in which the roles of the parties are reversed (a former master becomes the servant to his former servant, etc.). May also refer to the entities that seek retribution; divided into sha \'khon (those who bear malice towards our flesh) and rgyu \'khon (those who bear malice toward our property). Epstein, Dissertation 81, 106. Tucci, Religions 181. bu tsha lan chags sny

In [24]:
json_data = 

AttributeError: 'list' object has no attribute 'dumps'

In [16]:
def show_json_file(file_path: str):
    import json
    
    with open(file_path, "r") as file:
        # Read the file line by line
        data = []
        for line in file:
            try:
                json_obj = json.loads(line)
                data.append(json_obj)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
    
    # Print the JSON data
    print(json.dumps(data, indent=2))

 **Testing**

In [11]:
dataframe_to_json_file(df, "data.json")

In [17]:
show_json_file("data.json")

[
  {
    "tibetan": "\u0f63\u0f53\u0f0b\u0f46\u0f42\u0f66\u0f0d",
    "wylie": "lan chags/",
    "other": "lenchak",
    "definition": "karmic debt \u25a0 Karmic debt. A harmful effect that returns to someone who performed a negative action. \u25a0 In dialect, this can mean \u201cdifficulty\u201d or \u201chardship\u201d (dka' tshegs); it can also indicate a \u201cnegative condition\u201d or a \u201csource from which suffering arises\u201d (chags sgo). \u25a0 \u0f63\u0f53\u0f0b\u0f46\u0f42\u0f66\u0f0d \u25a0 a particular type of generalized karma which designates a relationship across lives in which the roles of the parties are reversed (a former master becomes the servant to his former servant, etc.). May also refer to the entities that seek retribution; divided into sha 'khon (those who bear malice towards our flesh) and rgyu 'khon (those who bear malice toward our property). Epstein, Dissertation 81, 106. Tucci, Religions 181. bu tsha lan chags snyog pa la byams par byas kyang mgu' 

In [4]:
df

,tibetan,wylie,other,definition
0,ལན་ཆགས།,lan chags/,lenchak,karmic debt ■ Karmic debt. A harmful effect th...
1,དག་པ་འཇིག་རྟེན་པའི་ཡེ་ཤེས,dag pa 'jig rten pa'i ye shes/,dakpa jikten peyeshé,དག་པ་འཇིག་རྟེན་པའི་ཡེ་ཤེས ■ A term used by Śān...
2,སྨན་བཅོས་སྐོར་བསྐྱོད,sman bcos skor bskyod/,menchö korkyö,སྨན་བཅོས་སྐོར་བསྐྱོད
3,སྨན་བཅོས་ཁང༌,sman bcos khang /,menchökhang,སྨན་བཅོས་ཁང༌ ■ clinic ■ clinic
4,སྨན་བཅོས་འཕྲོད་བསྟེན,sman bcos 'phrod bsten/,menchö tröten,སྨན་བཅོས་འཕྲོད་བསྟེན
...,...,...,...,...
183,སྨལ་པོ་ཟླ་ཕྱེད,smal po zla phyed/,melpo daché,སྨལ་པོ་ཟླ་ཕྱེད ■ [2169] ཉི་སྐར་ལྔ་དང་ཆུ་ཚོད་ལྔ...
184,སྨལ་པོའི་གདོང་ཆར,smal po'i gdong char/,melpö dongchar,སྨལ་པོའི་གདོང་ཆར ■ [2170] དབྱར་ཉི་ལྡོག་ནས་ལོག་...
185,སྨལ་ཟླ,smal zla/,menda,སྨལ་ཟླ ■ [2170] སྨལ་པོའི་ཟླ་བ་སྟེ་ཧོར་ཟླ་བཅུ་ག...
186,སྨས་པ་ལེན་པ,smas pa len pa/,mepa lenpa,སྨས་པ་ལེན་པ ■ [2170] ༼རྙིང༽ བརྙས་ཆོས་བྱེད་པ། ...


In [2]:
565 / 100

5.65

In [3]:
610 / 132

4.621212121212121

In [4]:
655 / 164

3.9939024390243905

In [5]:
700 / 196

3.5714285714285716

In [ ]:
# WHat is the average cost of strawberries per pound in Texas?
# Answer: 
